In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

import utils

country_aggregated_df = pd.read_csv('./Data/Gas_Trade_Flows_IEA_202310 - Data.csv')

### Process data

In [2]:
# Remove unnamed colum
country_aggregated_df = country_aggregated_df.drop(columns=['Unnamed: 2'])

# Remove the columns with flow data beteen Oct-08 and Aug-09
country_aggregated_df = country_aggregated_df.drop(columns=['Oct-08', 'Nov-08', 'Dec-08', 'Jan-09', 'Feb-09', 'Mar-09', 'Apr-09', 'May-09', 'Jun-09', 'Jul-09', 'Aug-09', 'Sep-09', 'Oct-09', 'Nov-09', 'Dec-09'])
country_aggregated_df.head()

,Borderpoint,Exit,Entry,MAXFLOW (Mm3/h),Jan-10,Feb-10,Mar-10,Apr-10,May-10,Jun-10,...,Jan-23,Feb-23,Mar-23,Apr-23,May-23,Jun-23,Jul-23,Aug-23,Sep-23,Oct-23
0,Adriatic LNG,Liquefied Natural Gas,Italy,1.10,602,581,657,622,608,439,...,687.0,654.0,850.0,714.0,809.0,620.0,823.0,693.0,754.0,745.0
1,Almeria,Algeria,Spain,1.07,0,0,0,0,0,0,...,760.0,659.0,569.0,619.0,698.0,448.0,767.0,696.0,709.0,863.0
2,Alveringem,Belgium,France,NaN,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alveringem,France,Belgium,0.37,0,0,0,0,0,0,...,644.0,616.0,185.0,240.0,392.0,396.0,195.0,50.0,394.0,371.0
4,Badajoz,Portugal,Spain,0.30,5,4,2,0,0,0,...,54.0,0.0,4.0,91.0,139.0,84.0,44.0,25.0,19.0,27.0


In [3]:
country_aggregated_df.shape

(228, 170)

### Countries in the data

In [4]:
unique_countries = set(country_aggregated_df['Exit'].unique()) | set(country_aggregated_df['Entry'].unique())

print("Data contains {} unique countries".format(len(unique_countries)))
print(unique_countries)


Data contains 45 unique countries
{'Norway', 'Belarus', 'Not Elsewhere Specified', 'Albania', 'Lithuania', 'Denmark', 'Moldova', 'Estonia', 'Luxembourg', 'Netherlands', 'Morocco', 'Czech Republic', 'Hungary', 'Iran', 'Switzerland', 'Greece', 'United Kingdom', 'Serbia', 'Latvia', 'North Macedonia', 'Ukraine', 'Bulgaria', 'Algeria', 'Libya', 'Liquefied Natural Gas', 'Sweden', 'Portugal', 'Georgia', 'Ireland', 'Italy', 'Belgium', 'Slovak Republic', 'Croatia', 'Poland', 'Spain', 'Austria', 'Finland', 'Republic of Türkiye', 'Tunisia', 'France', 'Slovenia', 'Russia', 'Germany', 'Romania', 'Isle of Man'}


In [5]:
print("Pure exporting countries: {}".format(set(country_aggregated_df['Exit'].unique()) - set(country_aggregated_df['Entry'].unique())))
print("Pure importing countries: {}".format(set(country_aggregated_df['Entry'].unique()) - set(country_aggregated_df['Exit'].unique())))

Pure exporting countries: {'Georgia', 'Algeria', 'Libya', 'Iran', 'Belarus', 'Not Elsewhere Specified'}
Pure importing countries: {'Sweden', 'Ireland', 'Moldova', 'Isle of Man', 'Luxembourg'}


In [8]:
# Retrieve columns with flow data 
mm_yyyy = country_aggregated_df.iloc[:,country_aggregated_df.columns.get_loc('Jan-10'):]

## Test configure grid for max-flow

In [9]:
graphs = utils.create_graphs_from_dataset(country_aggregated_df)

In [10]:
# Get most recent grid
oct_23 = graphs[-1]

In [11]:
# Display edge data
oct_23_edges = utils.get_edge_data(oct_23)
oct_23_edges

,Source,Target,Borderpoint,Max_Flow,Flow
0,Liquefied Natural Gas,Italy,Adriatic LNG,818.40,745.0
1,Liquefied Natural Gas,Italy,Livorno,468.72,203.0
2,Liquefied Natural Gas,Italy,Panigaglia,401.76,69.0
3,Liquefied Natural Gas,Italy,Piombino,372.00,198.0
4,Liquefied Natural Gas,Spain,Barcelona,1450.80,320.0
...,...,...,...,...,...
222,Belarus,Lithuania,Kotlovka,967.20,240.0
223,Belarus,Lithuania,Privalka,7.44,32.0
224,Not Elsewhere Specified,Ukraine,Not Elsewhere Specified,6161.00,0.0
225,Morocco,Spain,Tarifa,1220.16,0.0


In [12]:
# Display node data
oct_23_nodes = utils.get_node_data(oct_23)
oct_23_nodes

,Country,Total Demand,Summer Demand,Winter Demand,Max Production,Storage Deliverability,Storage Injection,Storage WGV,LNG Send-out,LNG Storage,Power Generation
0,Liquefied Natural Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Italy,718.480,271.875,394.777,95.000,2.916,1.699,195.844,707.000,617.000,44.215
2,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Spain,359.621,164.811,175.911,4.000,241.000,167.000,34.090,1.940,3.317,29.903
4,Belgium,154.113,60.583,81.648,NaN,170.000,88.000,9.130,669.000,566.000,6.915
5,France,429.707,138.442,276.604,24.000,2.488,1.131,134.351,1.358,1.340,12.893
6,Portugal,59.894,30.338,26.516,NaN,86.000,24.000,3.570,200.000,390.000,4.585
7,Estonia,3.770,1.133,2.178,NaN,NaN,NaN,NaN,NaN,NaN,110.000
8,Finland,11.833,4.560,5.924,1.000,NaN,NaN,NaN,146.000,NaN,1.780
9,Germany,813.921,229.220,531.200,174.000,6.806,4.262,254.673,NaN,NaN,31.808


In [13]:
# Identify edges with flow greater than max flow
print("Edges with flow exceeding capacity: ", oct_23_edges[oct_23_edges['Flow'] > oct_23_edges['Max_Flow']].size)

Edges with flow exceeding capacity:  55


In [14]:

oct_23 = utils.update_edge_capacities(oct_23)
oct_23_edges = utils.get_edge_data(oct_23)
print("Edges with flow exceeding capacity: ", oct_23_edges[oct_23_edges['Flow'] > oct_23_edges['Max_Flow']].size)

Edges with flow exceeding capacity:  0


## Convert multi-edge directed graph to directed graph
Required by max-flow algorithm

In [15]:
oct_23_digraph = utils.create_digraph_of(oct_23)